[<img align="left" width="140px" src="https://storage.googleapis.com/static.drivetrainhub.com/img/dh_logo_text_217x80.png"/>](https://drivetrainhub.com)

<div align="right">© 2020 Drivetrain Hub LLC</div>

# Geometry / Planetary Gears
---

**Authors**:  [Chad Glinsky](https://www.linkedin.com/in/chad-glinsky-a1840b13/)

**Description**:  Review of geometry for planetary gears and planetary gear meshes.

## Table of Contents

1. [Introduction](#Introduction)
1. [Layouts](#Layouts)
    1. [Nomenclature](#Nomenclature-1)
    1. [Mating Conditions](#Mating-Conditions)
    1. [Spur Example](#Example-1)
    1. [Helical Example](#Example-2)
1. [Gear Meshes](#Gear-Mesh)
    1. [Nomenclature](#Nomenclature-2)
    1. [Compatibility](#Compatibility)
    1. [Center Distances](#Center-Distances)
    1. [Kinematics](#Kinematics)
    1. [Phasing](#Phasing)
    1. [Example](#Example-2)
1. [References](#References)

#### Notebook imports and settings

In [40]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from math import pi, radians, degrees, isclose, sqrt, asin, floor
import numpy as np
import pandas as pd
from IPython.display import display, HTML

# notebook modules
# TODO develop module
# import planetary_gears as pg

# settings
FIGSIZE = (6, 6)  # size of plots
plt.rcParams.update({'font.size': 12})  # set default font size

In [3]:
# DEVELOPMENT USE: %autoreload 1
# PRODUCTION USE: %autoreload 0
%load_ext autoreload
%autoreload 1
%aimport helical_gears

$\LaTeX$ commands
$\newcommand{\inv}{\text{inv }}$
$\newcommand{\mm}{\text{mm}}$
$\newcommand{\deg}{^\circ}$
$\newcommand{\degt}{\text{deg}}$
$\newcommand{\perinch}{\text{in}^{-1}}$
$\newcommand{\mps}{\text{m/s}}$
$\newcommand{\rpm}{\text{rpm}}$

## Introduction

The geometry of planetary gears and gear meshes is reviewed here.  Planetary geartrains can be configured in several ways, with the most common architectures reviewed here.  Attractive characteristics of planetary geartrains include:

- Mechanical power is transferred between concentric bodies (ring, sun, carrier)
- Power density is high, providing great mechanical power in a small package
- Cylindrical involute gears are used to transmit motion
- Multiple power inputs or outputs can be defined

The visualization of a planetary geartrain clearly demonstrates its unique geometric characteristics.

<img src="./img/drivetrainhub_planetary_spur_gears.png" alt="Spur planetary geartrain" style="max-height: 350px; max-width: 100%"/>
<p style="text-align: center; font-weight: bold;">Planetary geartrain modeled with <a href="https://drivetrainhub.com/gears" style="text-decoration: none;">Gears App</a></p>


Most commonly, planetary geartrains have a sun gear (green), ring gear (blue), and multiple planet gears (pink) on a carrier (gray). Spur or helical gears are used for a planetary gearset, with each type having advantages.  This notebook reviews the unique geometric attributes of planetary gears and their significance to planetary gear mesh operation.

<div class="alert alert-block alert-info" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <b>The assembly of a planetary geartrain is dependent on various geometric attributes, such as tooth counts.</b>
    <br><br>
    The various conditions that must be satisfied for a planetary geartrain to successfully assemble are reviewed in this notebook.  To understand the geometry of the spur or helical gears that makeup a planetary geartrain, see the respective notebooks on gear geometry.
</div>

# TODO
- Concentric shaft layouts are often desirable for the application

Attributes specific to the gearing involved:

- Multiple gear meshes act on central bodies, with self-centering tendencies
- Precession of planets on a carrier 

## Layouts

This section reviews the geometric layout of planetary geartrains. A working example is provided at the end of the section.

### Nomenclature <a name="Nomenclature-1"></a>

This table provides a set of input parameters commonly used to define the geometry of planetary geartrains.

# FIXME
<table style="margin-left: 0">
    <tr>
        <th>Symbol</th>
        <th style="text-align:left">Description</th>
    </tr>
    <tr>
        <td style="text-align:left">$m_n$</td>
        <td style="text-align:left">Normal module</td>
    </tr>
    <tr>
        <td style="text-align:left">$\alpha_n$</td>
        <td style="text-align:left">Normal pressure angle</td>
    </tr>
    <tr>
        <td style="text-align:left">$z$</td>
        <td style="text-align:left">Number of teeth</td>
    </tr>
    <tr>
        <td style="text-align:left">$\beta$</td>
        <td style="text-align:left">Helix angle</td>
    </tr>
    <tr>
        <td style="text-align:left">$b$</td>
        <td style="text-align:left">Facewidth</td>
    </tr>
    <tr>
        <td style="text-align:left">$d_a$</td>
        <td style="text-align:left">Tip diameter</td>
    </tr>
    <tr>
        <td style="text-align:left">$d_f$</td>
        <td style="text-align:left">Root diameter</td>
    </tr>
    <tr>
        <td style="text-align:left">$s_n$</td>
        <td style="text-align:left">Normal tooth thickness</td>
    </tr>
    <tr>
        <td style="text-align:left">$\rho_f$</td>
        <td style="text-align:left">Tool tip radius</td>
    </tr>
</table>

It is also common to use addendum, dedendum, and profile shift coefficients of the basic rack or cutting tool instead of tip diameter, root diameter, and tooth thickness.  The former are better associated with the manufacturing process, while the latter are in reference to the manufactured geometry of a helical gear.

Notice tool tip radius is included in this set of input parameters.  It may seem out of place, but it is necessary to define since it relates directly to the manufacturing tooling responsible for the helical gear root geometry.  To understand this further, refer to the notebooks on gear tooling.

### Mating Conditions

To design a *valid* planetary geartrain, one that can be assembled and operate in real-world applications, numerous geometric conditions must be satisfied.

<div class="alert alert-block alert-info" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <strong>Geometric layouts are used to study the feasible design space for planetary geartrains.</strong>
    <br><br>
    As will be shown, certain combinations of tooth counts, planet count, and planet spacing are not possible.
</div>

#### Condition of Coaxiality
TODO

#### Condition of Neighboring
TODO

#### Assembly Condition
TODO


<div class="alert alert-block alert-warning" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <h3>Example | Spur Planetary Layout<a name="Example-1"></a></h3>
    <h4>GIVEN</h4>Input parameters to define a spur planetary geartrain layout.
    <h4>FIND</h4>Detailed geometric parameters of the spur planetary geartrain.
    <h4>SOLUTION</h4>See below.
</div>

# TODO move to formatted cell above

GIVEN:
1. Target tranmission ratio.
1. Target ring gear size.
1. Target number of planets.

COMPUTE:
1. Nominal pitch diameters based on target ratio.
1. Maximum number of planets to avoid interference.
1. Nearest tooth counts to achieve the target ratio.  This inherently sets the tooth module size.
1. Nearest-to-equal planet spacing.

In the next example, see how to target a tooth module size instead.

In [61]:
# TODO finish dev

# GIVEN
# -----
target_ratio_1v = 5  # omega_sun / omega_carrier (fixed ring)
target_ring_pitch_dia = -10
target_planets = 5  # FIXME del if not using

# EQUATIONS
# ---------
# i_1v = 1 - u
# ==> u = 1 - i_1v
# where u is the base ratio (fixed carrier)

# u = z3 / z1
# ==> z1 = z3 / u

# |z1 + z2| = |z3|
# ==> z1 = (|z3| - z1) * z3 / u

# SOLUTION - TARGET DIAMETERS
# ---------------------------
target_base_ratio = 1 - target_ratio_1v
target_sun_pitch_dia = target_ring_pitch_dia / target_base_ratio
target_planet_pitch_dia = (abs(target_ring_pitch_dia) - target_sun_pitch_dia) / 2
target_carrier_pitch_dia = target_sun_pitch_dia + target_planet_pitch_dia
target_center_distance = target_carrier_pitch_dia / 2

# SOLUTION - MAX PLANETS
# ----------------------
# l > tip_diameter_planet
# where l = 2 * a * sin(pi / N) 
# where l is the center distance between planets
# ==> N = pi / (asin(l / (2 * a)) + 2 * pi

tip_diameter_planet = target_planet_pitch_dia  # BE CAREFUL
number_of_planets_max = floor(pi / (asin(tip_diameter_planet / (2 * target_center_distance))))

if target_planets > number_of_planets_max:
    print(f"\x1b[31mWARNING: The target number of planets is too high, cannot exceed {number_of_planets_max}.\x1b[0m")  # red color formatting
    
number_of_planets = min([target_planets, number_of_planets_max])

# SOLUTION - TOOTH COUNTS
# -----------------------
# TODO determine the nearest tooth counts to satisfy the target diameters
# TODO there are infinite options depending on tooth size

# (z1 - z3) / N = k, where k is an integer
# Express z1 as a function of z3 and target ratio:
#   z1 = z3 / (1 - i_1v)
# Then rewrite equation as:
#   (z3 / (1 - i_1v) - z3) / N = k
# Solve z3:
#   z3 = k * N * (1 / i_1v - 1)
# Try increasing integers for k:

target_increment = number_of_planets * (1 / target_ratio_1v - 1)
for k in range(1, 70):
    z3 = k * target_increment
    z1 = z3 / target_base_ratio
    z2 = (abs(z3) - z1) / 2
    if isclose(z3, round(z3)):
        print(k, z3, z1, z2)


# DUMMY VALUES
module_normal = 1
helix_angle_deg = 0
number_of_teeth_sun = 0
number_of_teeth_planet = 0
number_of_teeth_ring = 0

# PLOT RESULTS
# ------------
# TODO plot

# DISPLAY RESULTS
# ---------------
cols = ['Description', 'Symbol', 'Value', 'Units']
data = [
    # INPUTS
    ['Target ratio', '$i_{1v_0}$', target_ratio_1v, '-'],
    ['Target base ratio', '$u$', target_base_ratio, '-'],
    ['Target pitch diameter, ring', '$d_{w3_0}$', target_ring_pitch_dia, '$\mm$'],
    # TARGETS
    ['---', '---', '---', '---'],
    ['Target pitch diameter, sun', '$d_{w1_0}$', target_sun_pitch_dia, '$\mm$'],
    ['Target pitch diameter, planets', '$d_{w2_0}$', target_planet_pitch_dia, '$\mm$'],
    ['Target pitch diameter, carrier', '$d_{wV_0}$', target_carrier_pitch_dia, '$\mm$'],
    ['Target center distance', '$a_0$', target_center_distance, '$\mm$'],
    ['Number of planets, max', '$N_\rm{max}$', number_of_planets_max, '-'],
    # OUTPUTS
    ['---', '---', '---', '---'],
    ['Normal module', '$m_n$', module_normal, '$\mm$'],
    ['Helix angle', '$\\beta$', helix_angle_deg, '$\degt$'],
    ['Number of teeth, sun', '$z_1$', number_of_teeth_sun, '-'],
    ['Number of teeth, planet', '$z_2$', number_of_teeth_planet, '-'],
    ['Number of teeth, ring', '$z_3$', number_of_teeth_ring, '-'],
]
df = pd.DataFrame(data=data, columns=cols)
display(HTML(df.to_html(index=False)))

5 -16.0 4.0 6.0
10 -32.0 8.0 12.0
15 -48.0 12.0 18.0
20 -64.0 16.0 24.0
25 -80.0 20.0 30.0
30 -96.0 24.0 36.0
35 -112.0 28.0 42.0
40 -128.0 32.0 48.0
45 -144.0 36.0 54.0
50 -160.0 40.0 60.0
55 -176.0 44.0 66.0
60 -192.0 48.0 72.0
65 -208.0 52.0 78.0


Description,Symbol,Value,Units
Target ratio,$i_{1v_0}$,5,-
Target base ratio,$u$,-4,-
"Target pitch diameter, ring",$d_{w3_0}$,-10,$\mm$
---,---,---,---
"Target pitch diameter, sun",$d_{w1_0}$,2.5,$\mm$
"Target pitch diameter, planets",$d_{w2_0}$,3.75,$\mm$
"Target pitch diameter, carrier",$d_{wV_0}$,6.25,$\mm$
Target center distance,$a_0$,3.125,$\mm$
"Number of planets, max",$N_\rm{max}$,4,-
---,---,---,---


In [64]:
# TODO develop example with a target tooth module size, with the overall envelope computed as a result.

In [63]:
# TODO develop example that allows targets for sum of profile shifts; also include a tooth module target


<div class="alert alert-block alert-warning" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <h3>Example | Helical Planetary Layout<a name="Example-2"></a></h3>
    <h4>GIVEN</h4>Input parameters to define a helical planetary geartrain layout.
    <h4>FIND</h4>Detailed geometric parameters of the helical planetary geartrain.
    <h4>SOLUTION</h4>See below.
</div>

In [65]:
# TODO example(s) for helical planetary layout

## Gear Meshes

# FIXME
This section reviews the properties of a helical gear mesh, both geometrically and kinematically.  A working example is provided at the end of the section.

### Nomenclature <a name="Nomenclature-2"></a>

This table provides a set of helical gear mesh parameters.

<table style="margin-left: 0">
    <tr>
        <th>Symbol</th>
        <th style="text-align:left">Description</th>
    </tr>
    <tr>
        <td style="text-align:left">$a$</td>
        <td style="text-align:left">Center distance</td>
    </tr>
    <tr>
        <td style="text-align:left">$a_0$</td>
        <td style="text-align:left">Reference center distance</td>
    </tr>
    <tr>
        <td style="text-align:left">$a_{j0}$</td>
        <td style="text-align:left">Theoretical center distance</td>
    </tr>
    <tr>
        <td style="text-align:left">$\alpha_w$</td>
        <td style="text-align:left">Working pressure angle</td>
    </tr>
    <tr>
        <td style="text-align:left">$\rm{P}$</td>
        <td style="text-align:left">Pitch point</td>
    </tr>
    <tr>
        <td style="text-align:left">$\rm{Y}$</td>
        <td style="text-align:left">Contact point</td>
    </tr>
    <tr>
        <td style="text-align:left">$\Sigma x$</td>
        <td style="text-align:left">Sum of profile shifts</td>
    </tr>
    <tr>
        <td style="text-align:left">$b$</td>
        <td style="text-align:left">Effective facewidth</td>
    </tr>
    <tr>
        <td style="text-align:left">$j_r$</td>
        <td style="text-align:left">Radial backlash</td>
    </tr>
    <tr>
        <td style="text-align:left">$j_{tt}$</td>
        <td style="text-align:left">Circumferential backlash</td>
    </tr>
    <tr>
        <td style="text-align:left">$j_{tn}$</td>
        <td style="text-align:left">Profile backlash</td>
    </tr>
    <tr>
        <td style="text-align:left">$j_{nn}$</td>
        <td style="text-align:left">Normal backlash</td>
    </tr>
    <tr>
        <td style="text-align:left">$\epsilon_\alpha$</td>
        <td style="text-align:left">Transverse contact ratio</td>
    </tr>
    <tr>
        <td style="text-align:left">$\epsilon_\beta$</td>
        <td style="text-align:left">Axial contact ratio</td>
    </tr>
    <tr>
        <td style="text-align:left">$\epsilon_\gamma$</td>
        <td style="text-align:left">Total contact ratio</td>
    </tr>
    <tr>
        <td style="text-align:left">$n_\alpha$</td>
        <td style="text-align:left">Transverse contact ratio remainder</td>
    </tr>
    <tr>
        <td style="text-align:left">$n_\beta$</td>
        <td style="text-align:left">Axial contact ratio remainder</td>
    </tr>
    <tr>
        <td style="text-align:left">$l_D$</td>
        <td style="text-align:left">Total length of contact lines</td>
    </tr>
</table>

# FIXME
The next table provides a set of helical gear mesh parameters specific to each gear, despite that certain parameters must be equal for each gear.

<table style="margin-left: 0">
    <tr>
        <th>Symbol</th>
        <th style="text-align:left">Description</th>
    </tr>
    <tr>
        <td style="text-align:left">$d_w$</td>
        <td style="text-align:left">Pitch diameter</td>
    </tr>
    <tr>
        <td style="text-align:left">$p_t$</td>
        <td style="text-align:left">Transverse pitch</td>
    </tr>
    <tr>
        <td style="text-align:left">$p_n$</td>
        <td style="text-align:left">Normal pitch</td>
    </tr>
    <tr>
        <td style="text-align:left">$p_a$</td>
        <td style="text-align:left">Axial pitch</td>
    </tr>
    <tr>
        <td style="text-align:left">$p_{bt}$</td>
        <td style="text-align:left">Transverse base pitch</td>
    </tr>
    <tr>
        <td style="text-align:left">$p_{bn}$</td>
        <td style="text-align:left">Normal base pitch</td>
    </tr>
    <tr>
        <td style="text-align:left">$p_{ba}$</td>
        <td style="text-align:left">Axial base pitch</td>
    </tr>
    <tr>
        <td style="text-align:left">$c_a$</td>
        <td style="text-align:left">Tip clearance</td>
    </tr>
    <tr>
        <td style="text-align:left">$c_f$</td>
        <td style="text-align:left">Bottom clearance</td>
    </tr>
    <tr>
        <td style="text-align:left">$j_\theta$</td>
        <td style="text-align:left">Angular backlash</td>
    </tr>
    <tr>
        <td style="text-align:left">$\alpha_{t\text{Y}}$</td>
        <td style="text-align:left">Transverse pressure angle at contact point</td>
    </tr>
    <tr>
        <td style="text-align:left">$\alpha_{t\text{SAP}}$</td>
        <td style="text-align:left">Transverse pressure angle at SAP</td>
    </tr>
    <tr>
        <td style="text-align:left">$\alpha_{t\text{EAP}}$</td>
        <td style="text-align:left">Transverse pressure angle at EAP</td>
    </tr>
    <tr>
        <td style="text-align:left">$d_\rm{SAP}$</td>
        <td style="text-align:left">SAP diameter</td>
    </tr>
    <tr>
        <td style="text-align:left">$d_\rm{EAP}$</td>
        <td style="text-align:left">EAP diameter</td>
    </tr>
    <tr>
        <td style="text-align:left">$\psi_\rm{SAP}$</td>
        <td style="text-align:left">SAP roll angle</td>
    </tr>
    <tr>
        <td style="text-align:left">$\psi_\rm{EAP}$</td>
        <td style="text-align:left">EAP roll angle</td>
    </tr>
    <tr>
        <td style="text-align:left">$v_w$</td>
        <td style="text-align:left">Pitch line velocity</td>
    </tr>
</table>

where SAP and EAP are the start and end of active profile, respectively, as explained later.

<div class="alert alert-block alert-info" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    For each gear in a gear pair, the subscripts <sub>$1$</sub> and <sub>$2$</sub> are used for the respective gear parameters.  For example, $d_{w1}$ is the pitch diameter of <i>gear 1</i>.
</div>

### Center Distances

# FIXME
The center distance of a helical gear pair is the radial distance between the rotational axis of each gear.  Three types of center distances are classified:

1. Reference center distance
2. Theoretical center distance
3. Working center distance

#### Reference center distance

Center distance of a gear pair without profile shifts or backlash.  Also known as *null center distance*.  For this condition, working pressure angle is equal to the reference pressure angle, $\alpha_w = \alpha_t$, and therefore the reference center distance is calculated as:

$$a_0 = \frac{d_1 + d_2}{2} = \frac{m_n (z_1 + z_2)}{2 \cos\beta}$$

#### Theoretical center distance

Center distance of a gear pair without backlash, but gears may have profile shift.  If the sum of profile shifts is zero, $\Sigma x = 0$, the theoretical center distance equals the null center distance, and working pressure angle equals the transverse reference pressure angle.

$$a_{j0} = a_0 \frac{\cos\alpha_t}{\cos\alpha_w}$$

where the specified working pressure angle, $\alpha_w$, must be for the condition of zero backlash.

#### Center distance

*Actual* center distance of an assembled gear pair.  It can be calculated from the pitch diameters, which are simply related by the number of teeth, as explained in the kinematics section.

$$a = \frac{d_{w1} + d_{w2}}{2}$$

Influence of profile shifts when backlash is zero, i.e. when *actual* center distance equals the theoretical center distance:

- $\Sigma x = 0 \Rightarrow a = a_0 \text{ and } \alpha_w = \alpha_t$
- $\Sigma x < 0 \Rightarrow a < a_0 \text{ and } \alpha_w < \alpha_t$
- $\Sigma x > 0 \Rightarrow a > a_0 \text{ and } \alpha_w > \alpha_t$

<div class="alert alert-block alert-info" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <b>Involute tooth profiles of helical gears allow for center distance variation without affecting the nominal transmission ratio or conjugate action.</b>
    <br><br>
    Actual meshing behavior may be affected since tooth stiffness and conjugate relief of microgeometry modifications could change with center distance.
</div>

### Kinematics

# FIXME
The kinematics of a helical gear pair are considered at two degrees of fidelity:

1. Kinematics of the gear bodies
2. Kinematics of the contacting gear teeth

The kinematics of a gear body can be expressed with a single degree of freedom (DOF), the angular velocity about its axis of rotation.  This fidelity is sufficient unless analyzing the dynamics of a fully elastic system, in which case a 6-DOF mathematical model is recommended.

If the local z-axis is denoted as the axis of rotation for a helical gear, its angular velocity can be expressed as:

$$
\vec{\omega} 
= \begin{bmatrix} \omega_x \\ \omega_y \\ \omega_z \end{bmatrix} 
= \begin{bmatrix} 0 \\ 0 \\ \omega_z \end{bmatrix} 
$$

#### Pitch Line Velocity

The pitch line velocity refers to the linear velocity at the gear mesh pitch point.  Pitch line velocity is important to consider for gear lubrication and dynamics, which can influence stress, wear, and noise.

<img src="./img/drivetrainhub_pitch_line_speed.svg" alt="Pitch line speed of a helical gear pair" style="height: 200px; max-width: 100%"/>
<p style="text-align: center; font-weight: bold;">Pitch line speed of a helical gear pair</p>

Pure rolling motion occurs at the pitch point of meshing helical gears and thus an equation for pitch line velocity is:

$$\vec{v}_w = \vec{r}_w \times  \vec{\omega}$$

where $\vec{v}_w = \vec{v}_\rm{P1,2}$ in the figure.

#### Transmission Ratio

Nominally, a constant ratio of angular velocities exists for the gears in a helical gear mesh, known as the *transmission ratio* or *gear mesh ratio*.  The ratio can be derived by considering the instantaneous linear velocity at the pitch point, where both gears are in pure roll.

$$\vec{r}_{w1} \times  \vec{\omega}_1 = \vec{r}_{w2} \times  \vec{\omega}_2$$

By solving this equation for the ratio of angular velocities, the gear mesh transmission ratio is defined as:

$$i = \frac{\omega_1}{\omega_2} = \frac{r_{w2}}{r_{w1}} = \frac{z_2}{z_1}$$

Direction of rotation depends on the gear mesh being external-external or internal-external type.

<img src="./img/drivetrainhub_gear_mesh_rotational_directions.svg" alt="Rotational directions of helical gear meshes" style="height: 250px; max-width: 100%"/>
<p style="text-align: center; font-weight: bold;">Rotational directions of helical gear meshes</p>
    
<div class="alert alert-block alert-danger" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <b>Real helical gear meshes do not operate with a constant ratio of angular velocities at all times.</b>
    <br><br>
    Actual velocities fluctuate about the transmission ratio defined here because true conjugate action is never achieved.  This is due to manufactured gears being imperfect and gear tooth loads causing deflections of the involute profile.  The transmission ratio, $i$, should be considered the <i>nominal</i> or <i>mean</i> ratio of angular velocities.
</div>

<div class="alert alert-block alert-info" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    The kinematics for a system of rotationally coupled bodies, e.g. meshing helical gears, is formulated in detail in our notebooks on rotational mechanics.
</div>

<div class="alert alert-block alert-warning" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <h3>Example | Planetary Gear Meshes<a name="Example-2"></a></h3>
    <h4>GIVEN</h4>Input parameters to define the spur gear meshes of a planetary geartrain.
    <h4>FIND</h4>Geometry, contact, and kinematic properties of planetary spur gear meshes.
    <h4>SOLUTION</h4>See below.
</div>

In [5]:
# FIXME update for planetary geartrain

# GIVEN
# -----
# common data
module_normal = 1
pressure_angle_normal_deg = 20
helix_angle_deg = 15
center_distance = 27.5
input_speed_rpm = 100

# gear 1 data
number_of_teeth1 = 17
helix_hand1 = 'right'
basic_rack_addendum_coefficient1 = 1
basic_rack_dedendum_coefficient1 = 1.25
tool_tip_radius_coefficient1 = 0.38
profile_shift_coefficient1 = 0.2
facewidth1 = 10

# gear 2 data
number_of_teeth2 = 35
helix_hand2 = 'left'
basic_rack_addendum_coefficient2 = 1.0
basic_rack_dedendum_coefficient2 = 1.25
tool_tip_radius_coefficient2 = 0.38
profile_shift_coefficient2 = -0.1
facewidth2 = 9

# SOLUTION
# -------- 
# conversions
helix_angle = radians(helix_angle_deg)
pressure_angle_normal = radians(pressure_angle_normal_deg)

helix_sign1 = 1 if helix_hand1 == 'right' else -1
helix_sign2 = 1 if helix_hand2 == 'right' else -1
helix_angle1 = radians(helix_angle_deg) * helix_sign1
helix_angle2 = radians(helix_angle_deg) * helix_sign2
basic_rack_addendum1 = basic_rack_addendum_coefficient1 * module_normal
basic_rack_addendum2 = basic_rack_addendum_coefficient2 * module_normal
basic_rack_dedendum1 = basic_rack_dedendum_coefficient1 * module_normal
basic_rack_dedendum2 = basic_rack_dedendum_coefficient2 * module_normal
profile_shift1 = profile_shift_coefficient1 * module_normal
profile_shift2 = profile_shift_coefficient2 * module_normal

# gear parameters
module_transverse = hg.module_transverse_fcn(module_normal, helix_angle)
pressure_angle_transverse = hg.pressure_angle_transverse_fcn(pressure_angle_normal, helix_angle)
theoretical_pitch_diameter1 = hg.theoretical_pitch_diameter_fcn(module_transverse, number_of_teeth1)
theoretical_pitch_diameter2 = hg.theoretical_pitch_diameter_fcn(module_transverse, number_of_teeth2)
base_diameter1 = hg.base_diameter_fcn(theoretical_pitch_diameter1, pressure_angle_transverse)
base_diameter2 = hg.base_diameter_fcn(theoretical_pitch_diameter2, pressure_angle_transverse)
tip_diameter1 = hg.tip_diameter_fcn(theoretical_pitch_diameter1, basic_rack_addendum1, profile_shift1)
tip_diameter2 = hg.tip_diameter_fcn(theoretical_pitch_diameter2, basic_rack_addendum2, profile_shift2)
root_diameter1 = hg.root_diameter_fcn(theoretical_pitch_diameter1, basic_rack_dedendum1, profile_shift1)
root_diameter2 = hg.root_diameter_fcn(theoretical_pitch_diameter2, basic_rack_dedendum2, profile_shift2)
helix_angle_base1 = hg.helix_angle_arbitrary_fcn(base_diameter1, helix_angle, theoretical_pitch_diameter1)
helix_angle_base2 = hg.helix_angle_arbitrary_fcn(base_diameter2, helix_angle, theoretical_pitch_diameter2)

# CHECK INPUTS
# ------------
assert helix_hand1 != helix_hand2

if center_distance > (tip_diameter1 + tip_diameter2) / 2:
    raise ValueError('Center distance is too large, try reducing it or change the gear geometry.')
    
if center_distance < (root_diameter1 + root_diameter2) / 2:
    raise ValueError('Center distance is too small, try increasing it or change the gear geometry.')

# pressure angles and center distances
working_pressure_angle = hg.working_pressure_angle_fcn(center_distance, base_diameter1, base_diameter2)
working_pressure_angle_theoretical = hg.working_pressure_angle_theoretical_fcn(profile_shift_coefficient1, profile_shift_coefficient2, number_of_teeth1, number_of_teeth2, pressure_angle_normal, pressure_angle_transverse)
center_distance_reference = hg.center_distance_reference_fcn(module_normal, number_of_teeth1, number_of_teeth2, helix_angle)
center_distance_theoretical = hg.center_distance_theoretical_fcn(center_distance_reference, working_pressure_angle_theoretical, pressure_angle_transverse)

# rotational speeds
transmission_ratio = hg.transmission_ratio_fcn(number_of_teeth1, number_of_teeth2)
rpm1 = input_speed_rpm
rpm2 = rpm1 / transmission_ratio
angular_velocity1 = rpm1 * pi / 30
angular_velocity2 = rpm2 * pi / 30
working_pitch_diameter1, working_pitch_diameter2 = hg.pitch_diameters_fcn(center_distance, number_of_teeth1, number_of_teeth2)

# pitches (same for both gears)
pitch_transverse = hg.pitch_transverse_fcn(theoretical_pitch_diameter1, number_of_teeth1)
pitch_normal = hg.pitch_normal_fcn(module_normal)
pitch_axial = hg.pitch_axial_fcn(module_normal, helix_angle)
base_pitch_transverse = hg.pitch_transverse_fcn(base_diameter1, number_of_teeth1)
base_pitch_normal = hg.base_pitch_normal_fcn(module_normal, pressure_angle_normal)
base_pitch_axial = hg.base_pitch_axial_fcn(module_normal, pressure_angle_normal, helix_angle_base1)

# clearances
tip_clearance1 = hg.tip_clearance_fcn(center_distance, tip_diameter1, root_diameter2)
tip_clearance2 = hg.tip_clearance_fcn(center_distance, tip_diameter2, root_diameter1)
bottom_clearance1 = hg.bottom_clearance_fcn(center_distance, root_diameter1, tip_diameter2)
bottom_clearance2 = hg.bottom_clearance_fcn(center_distance, root_diameter2, tip_diameter1)
backlash_radial = hg.backlash_radial_fcn(center_distance, center_distance_theoretical)
backlash_circumferential = hg.backlash_circumferential_fcn(backlash_radial, working_pressure_angle)
backlash_profile = hg.backlash_profile_fcn(backlash_circumferential, working_pressure_angle)
backlash_normal = hg.backlash_normal_fcn(backlash_profile, helix_angle_base)
backlash_angular1 = hg.backlash_angular_fcn(backlash_circumferential, working_pitch_diameter1)
backlash_angular2 = hg.backlash_angular_fcn(backlash_circumferential, working_pitch_diameter2)

# SAP & EAP
pressure_angle_transverse_eap1 = hg.pressure_angle_transverse_arbitrary_fcn(base_diameter1, tip_diameter1)
pressure_angle_transverse_eap2 = hg.pressure_angle_transverse_arbitrary_fcn(base_diameter2, tip_diameter2)
pressure_angle_transverse_sap1 = hg.pressure_angle_transverse_contact_fcn(working_pressure_angle, number_of_teeth1, number_of_teeth2, pressure_angle_transverse_eap2)
pressure_angle_transverse_sap2 = hg.pressure_angle_transverse_contact_fcn(working_pressure_angle, number_of_teeth2, number_of_teeth1, pressure_angle_transverse_eap1)
diameter_sap1 = hg.pressure_angle_transverse_to_diameter(base_diameter1, pressure_angle_transverse_sap1)
diameter_sap2 = hg.pressure_angle_transverse_to_diameter(base_diameter2, pressure_angle_transverse_sap2)
diameter_eap1 = hg.pressure_angle_transverse_to_diameter(base_diameter1, pressure_angle_transverse_eap1)
diameter_eap2 = hg.pressure_angle_transverse_to_diameter(base_diameter2, pressure_angle_transverse_eap2)
roll_angle_sap1 = hg.diameter_to_roll_angle(base_diameter1, diameter_sap1)
roll_angle_sap2 = hg.diameter_to_roll_angle(base_diameter2, diameter_sap2)
roll_angle_eap1 = hg.diameter_to_roll_angle(base_diameter1, diameter_eap1)
roll_angle_eap2 = hg.diameter_to_roll_angle(base_diameter2, diameter_eap2)

# contact
facewidth_effective = min([facewidth1, facewidth2])
contact_ratio_transverse = hg.contact_ratio_transverse_fcn(working_pressure_angle, number_of_teeth1, number_of_teeth2, pressure_angle_transverse_eap1, pressure_angle_transverse_eap2)
contact_ratio_axial = hg.contact_ratio_axial_fcn(facewidth_effective, helix_angle, module_normal)
contact_ratio_total = hg.contact_ratio_total_fcn(contact_ratio_transverse, contact_ratio_axial)
contact_plane_length = hg.contact_plane_length_fcn(working_pressure_angle, base_diameter1, base_diameter2, pressure_angle_transverse_eap1, pressure_angle_transverse_eap2)
contact_lines_length_mean = hg.contact_lines_length_mean_fcn(facewidth_effective, contact_ratio_transverse, helix_angle_base)
contact_lines_length_min = hg.contact_lines_length_min_fcn(contact_lines_length_mean, contact_ratio_transverse, contact_ratio_axial)
contact_lines_length_delta = (contact_lines_length_mean - contact_lines_length_min) / contact_lines_length_mean

# kinematics
pitch_line_velocity = hg.pitch_line_velocity_fcn(working_pitch_diameter1, angular_velocity1)
sliding_velocity_sap1, sliding_velocity_eap2 = hg.sliding_velocity_fcn(base_diameter1, base_diameter2, pressure_angle_transverse_sap1, pressure_angle_transverse_eap2, angular_velocity1, angular_velocity2)
sliding_velocity_sap2, sliding_velocity_eap1 = hg.sliding_velocity_fcn(base_diameter1, base_diameter2, pressure_angle_transverse_sap2, pressure_angle_transverse_eap1, angular_velocity1, angular_velocity2)
specific_sliding_sap1, specific_sliding_eap2 = hg.specific_sliding_fcn(base_diameter1, base_diameter2, pressure_angle_transverse_sap1, pressure_angle_transverse_eap2, angular_velocity1, angular_velocity2)
specific_sliding_sap2, specific_sliding_eap1 = hg.specific_sliding_fcn(base_diameter2, base_diameter1, pressure_angle_transverse_sap2, pressure_angle_transverse_eap1, angular_velocity2, angular_velocity1)

# CHECK OUTPUTS
# -------------
assert isclose(helix_angle_base1, helix_angle_base2)
diametral_clearance_min = 0.25 * module_normal

if tip_clearance1 < diametral_clearance_min:
    print("\x1b[31mWARNING: Tip clearance of gear 1 is lower than recommended.\x1b[0m")  # red color formatting
    
if tip_clearance2 < diametral_clearance_min:
    print("\x1b[31mWARNING: Tip clearance of gear 2 is lower than recommended.\x1b[0m")  # red color formatting
    
if bottom_clearance1 < diametral_clearance_min:
    print("\x1b[31mWARNING: Bottom clearance of gear 1 is lower than recommended.\x1b[0m")  # red color formatting
    
if bottom_clearance2 < diametral_clearance_min:
    print("\x1b[31mWARNING: Bottom clearance of gear 2 is lower than recommended.\x1b[0m")  # red color formatting

# DISPLAY RESULTS
# ---------------
cols = ['Description', 'Symbol', 'Gear 1', 'Common', 'Gear 2', 'Units']
data = [
    # GEAR DATA
    ['Normal module', '$m_n$', '-', module_normal, '-', '$\mm$'],
    ['Normal pressure angle', '$\\alpha_n$', '-', pressure_angle_normal_deg, '-', '$\degt$'],
    ['Helix angle', '$\\beta$', '-', helix_angle_deg, '-', '$\degt$'],
    ['Base helix angle', '$\\beta_b$', '-', degrees(helix_angle_base1), '-', '$\degt$'],
    ['Number of teeth', '$z$', number_of_teeth1, '-', number_of_teeth2, '-'],
    ['Addendum coefficient', '$h_a^*$', basic_rack_addendum_coefficient1, '-', basic_rack_addendum_coefficient2, '-'],
    ['Dedendum coefficient', '$h_f^*$', basic_rack_dedendum_coefficient1, '-', basic_rack_dedendum_coefficient2, '-'],
    ['Tool tip radius coefficient', '$\\rho_f^*$', tool_tip_radius_coefficient1, '-', tool_tip_radius_coefficient2, '-'],
    ['Profile shift coefficient', '$x^*$', profile_shift_coefficient1, '-', profile_shift_coefficient2, '-'],
    ['Theoretical pitch diameter', '$d$', theoretical_pitch_diameter1, '-', theoretical_pitch_diameter2, '$\mm$'],
    ['Base diameter', '$d_b$', base_diameter1, '-', base_diameter2, '$\mm$'],
    ['Root diameter', '$d_f$', root_diameter1, '-', root_diameter2, '$\mm$'],
    ['Tip diameter', '$d_a$', tip_diameter1, '-', tip_diameter2, '$\mm$'],
    ['Facewidth', '$b$', facewidth1, '-', facewidth2, '$\mm$'],
    ['Effective facewidth', '$b$', '-', facewidth_effective, '-', '$\mm$'],
    # CD, RATIO, SPEED
    ['Reference center distance', '$a_0$', '-', center_distance_reference, '-', '$\mm$'],
    ['Theoretical center distance', '$a_{j0}$', '-', center_distance_theoretical, '-', '$\mm$'],
    ['Actual center distance', '$a$', '-',  center_distance, '-', '$\mm$'],
    ['Working pressure angle', '$\\alpha_w$', '-', degrees(working_pressure_angle), '-', '$\degt$'],
    ['Transmission ratio', '$i$', '-', transmission_ratio, '-', '-'],
    ['Rotational speed', '$\omega$', rpm1, '-', rpm2, '$\rpm$'],
    ['Pitch diameter', '$d_w$', working_pitch_diameter1, '-', working_pitch_diameter2, '$\mm$'],
    # PITCHES
    ['Transverse pitch', '$p_t$', '-', pitch_transverse, '-', '$\mm$'],
    ['Normal pitch', '$p_n$', '-', pitch_normal, '-', '$\mm$'],
    ['Axial pitch', '$p_a$', '-', pitch_axial, '-', '$\mm$'],
    ['Transverse base pitch', '$p_{bt}$', '-', base_pitch_transverse, '-', '$\mm$'],
    ['Normal base pitch', '$p_{bn}$', '-', base_pitch_normal, '-', '$\mm$'],
    ['Axial base pitch', '$p_{ba}$', '-', base_pitch_axial, '-', '$\mm$'],
    # CLEARANCES
    ['Radial backlash', '$j_r$', '-', backlash_radial, '-', '$\mm$'],
    ['Circumferential backlash', '$j_{tt}$', '-', backlash_circumferential, '-', '$\mm$'],
    ['Profile backlash', '$j_{tn}$', '-', backlash_profile, '-', '$\mm$'],
    ['Normal backlash', '$j_{nn}$', '-', backlash_normal, '-', '$\mm$'],
    ['Angular backlash', '$j_\\theta$', backlash_angular1, '-', backlash_angular2, '$\degt$'],
    ['Tip clearance', '$c_a$', tip_clearance1, '-', tip_clearance2, '$\mm$'],
    ['Bottom clearance', '$c_f$', bottom_clearance1, '-', bottom_clearance2, '$\mm$'],
    # SAP & EAP
    ['Transverse pressure angle at SAP', '$\\alpha_\\text{tSAP}$', degrees(pressure_angle_transverse_sap1), '-', degrees(pressure_angle_transverse_sap2), '$\degt$'],
    ['Transverse pressure angle at EAP', '$\\alpha_\\text{tEAP}$', degrees(pressure_angle_transverse_eap1), '-', degrees(pressure_angle_transverse_eap2), '$\degt$'],
    ['Roll angle at SAP', '$\\psi_\\text{SAP}$', degrees(roll_angle_sap1), '-', degrees(roll_angle_sap2), '$\degt$'],
    ['Roll angle at EAP', '$\\psi_\\text{EAP}$', degrees(roll_angle_eap1), '-', degrees(roll_angle_eap2), '$\degt$'],
    ['Diameter at SAP', '$d_\\text{SAP}$', diameter_sap1, '-', diameter_sap2, '$\mm$'],
    ['Diameter at EAP', '$d_\\text{EAP}$', diameter_eap1, '-', diameter_eap2, '$\mm$'],
    # CONTACT
    ['Transverse contact ratio', '$\\epsilon_\\alpha$', '-', contact_ratio_transverse, '-', '-'],
    ['Axial contact ratio', '$\\epsilon_\\beta$', '-', contact_ratio_axial, '-', '-'],
    ['Total contact ratio', '$\\epsilon_\\gamma$', '-', contact_ratio_total, '-', '-'],
    ['Contact plane length', '$\\overline{\\text{AE}}$', '-', contact_plane_length, '-', '$\mm$'],
    ['Mean total length of contact lines', '$l_{D\\text{mean}}$', '-', contact_lines_length_mean, '-', '$\mm$'],
    ['Minimum total length of contact lines', '$l_{D\\text{min}}$', '-', contact_lines_length_min, '-', '$\mm$'],
    ['Variation of total length of contact lines', '$\\Delta l_D$', '-', 100 * contact_lines_length_delta, '-', '$\%$'],
    # KINEMATICS
    ['Pitch line velocity', '$v_w$', '-', pitch_line_velocity, '-', '-'],
    ['Sliding velocity at SAP', '$v_{R,\\text{SAP}}$', sliding_velocity_sap1, '-', sliding_velocity_sap2, '$\mps$'],
    ['Sliding velocity at EAP', '$v_{R,\\text{EAP}}$', sliding_velocity_eap1, '-', sliding_velocity_eap2, '$\mps$'],
    ['Specific sliding at SAP', '$\\vartheta_\\text{SAP}$', specific_sliding_sap1, '-', specific_sliding_sap2, '-'],
    ['Specific sliding at EAP', '$\\vartheta_\\text{EAP}$', specific_sliding_eap1, '-', specific_sliding_eap2, '-'],
]
df = pd.DataFrame(data=data, columns=cols)
display(HTML(df.to_html(index=False)))

Description,Symbol,Gear 1,Common,Gear 2,Units
Normal module,$m_n$,-,1,-,$\mm$
Normal pressure angle,$\alpha_n$,-,20,-,$\degt$
Helix angle,$\beta$,-,15,-,$\degt$
Base helix angle,$\beta_b$,-,14.0761,-,$\degt$
Number of teeth,$z$,17,-,35,-
Addendum coefficient,$h_a^*$,1,-,1,-
Dedendum coefficient,$h_f^*$,1.25,-,1.25,-
Tool tip radius coefficient,$\rho_f^*$,0.38,-,0.38,-
Profile shift coefficient,$x^*$,0.2,-,-0.1,-
Theoretical pitch diameter,$d$,17.5997,-,36.2347,$\mm$


---
<div class="alert alert-block alert-info" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <h3>Model Gears</h3><br>
    <a href="https://drivetrainhub.com/gears" style="font-weight: bold">Gears App</a> online software is used to accurately model, analyze, and build planetary geartrains entirely in your web browser.
</div>

<div class="alert alert-block alert-info" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <h3>Learn More</h3><br>
    <a href="https://drivetrainhub.com/notebooks/" style="font-weight: bold">Notebook Series</a> is free to learn and contribute knowledge about gears, such as geometry, manufacturing, strength, and more.
</div>

<div class="alert alert-block alert-info" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <h3>Edit Notebook</h3><br>
    <a href="https://github.com/drivetrainhub/notebooks/" style="font-weight: bold">GitHub repos</a> are used to publicly host our notebooks, allowing anyone to view and propose edits.
</div>

## References

1. [Gears and Gear Drives, 1st Edition. Damir Jelaska](https://www.wiley.com/en-us/Gears+and+Gear+Drives-p-9781119941309)
1. Handbook of Practical Gear Design and Manufacture, 1st Edition. Darle W. Dudley
1. ANSI/AGMA 1010-F14, Appearance of Gear Teeth - Terminology of Wear and Failure
1. Cheng, Harry H., "Derivation of the Explicit Solution of the Inverse Involute Function and its Application in gear Tooth Geometry", Journal of Applied Mechanisms and Robotics, 1996
1. [Wolfram MathWorld - Helix](http://mathworld.wolfram.com/Helix.html)
1. [Wikipedia - Helix](https://en.wikipedia.org/wiki/Helix)